In [ ]:
import pandas as pd
import random as rn
import matplotlib.pyplot as plt
import scipy.stats as st
from gurobipy import *
import time
import numpy as np
import seaborn as sn

In [ ]:
path = "D:/demand_profile.csv"

# getting demand profile i.e hourly demand for each customer
demand_profile = pd.read_csv(path,header=None);
demand_profile = demand_profile.iloc[:50,:].copy()
n_cust,n_t = demand_profile.shape;


#getting daily demand of the customers
demand = demand_profile.sum(axis=1).values;

#getting mean hourly demand of each customer
mean_hourly = np.rint(demand_profile.mean(axis=1).values)

#Getting Min, Max, Avg and Standard Hourly Mean Demand
print("\nMinimum Hourly Mean Demand= ",round(np.min(mean_hourly),ndigits=0),
        "\nMaximum Hourly Mean Demand= ",round(np.max(mean_hourly),ndigits=0),
        "\nMedian Hourly Mean Demand= ",round(np.median(mean_hourly),ndigits=0),
        "\nMode Hourly Mean Demand= ",st.mode(mean_hourly),
        "\nAverage Hourly Mean Demand= ",round(np.mean(mean_hourly),ndigits=0),
        "\nStandard Hourly Mean Demand= ",round(np.std(mean_hourly),ndigits=0))

# Getting Fuel Data
path = "D:/fuel_properties.csv"
data_f = pd.read_csv(path)
data_f['Eff- kWh/$/CO2'] = data_f['kWh/Unit Fuel']*100/data_f['Unit Price']/data_f['CO2 lbs/Fuel Unit']
data_f['Eff- kWh/$'] = data_f['kWh/Unit Fuel']*100/data_f['Unit Price']

#getting length of the data
n_fuel = len(data_f.index);

alp_rate = list(data_f["kWh/Unit Fuel"].values) # kWh/fuel source
beta_rate = list(data_f["CO2 lbs/Fuel Unit"].values) # CO2 lbs/fuel source

fuel_supply=[]
capacities = [.50, .60, .20]
for i in range(n_fuel):
    fuel_supply.append(round(demand.sum()*(capacities[i]/alp_rate[i]),ndigits=0))

In [ ]:
#other fuel properties

cost_gamma = [10,10,10,10,15,15,15,15,20,20,20,20,20,10,10,10,10,10,10,15,15,15,10,10]


emission_penalty = 10;  #Penalty assigned to CO2 emisisons for getting weighted profits

cost_fuel = list(data_f["Unit Price"].values) #cost of fuel per unit consumption

cost_ramp = 0.50; #ramping up or down cost

caligraphy_n = 24; #maximum number of price change allowed


epsilon_rate = round(np.median(mean_hourly),ndigits=0); #maximum ramping allowed at no cost


#inconvenience cost to the customer for demand shift to other time
cost_xi=pd.DataFrame(0,index=np.arange(n_cust),columns=np.arange(n_t));

# demand shift capacity of customer
omega_rate=pd.DataFrame(0,index=np.arange(n_cust),columns=np.arange(n_t));

for i in range(n_cust):
    n = demand_profile.iloc[i,:].max()
    m = demand_profile.iloc[i,:].mean()
    for t in range(n_t):
        r = demand_profile.iloc[i,t]
        cost_xi.iloc[i,t] = abs(round((1-(r/n))/5,ndigits=2))
        omega_rate.iloc[i,t] = abs(round(r*(1-r/n)*1,ndigits=0))

In [ ]:
gur_env=Env(empty=False);
gur_env.setParam('MIPGap',0.01);
gur_env.setParam('OutputFlag',1);

In [ ]:
times=range(n_t)
fuels=range(n_fuel)
customers=range(n_cust)

In [ ]:
def with_emission(n_t,n_fuel,n_cust,alp_rate,fuel_supply,epsilon_rate,BIGM_price,caligraphy_n,beta_rate,
                  demand_profile,demand,threshold,omega_rate,cost_xi,cost_gamma,BIGM_kkt1,BIGM_kkt2,BIGM_kkt3,
                  cost_fuel,cost_ramp,emission_penalty,data=data_f):
    
    """Function to Write Model with Carbon Emissions Minimized"""
    
    M1=Model(env=gur_env, name='M1')

    #Defining Leaders' variables
    # Energy produced at a given hour 
    X = M1.addVars(times,lb=0.0,ub=GRB.INFINITY,vtype=GRB.CONTINUOUS,name="X")
    # Units of electricity consumed for the different fuel sources at a given hour
    Y = M1.addVars(fuels,times,lb=0.0,ub=GRB.INFINITY,vtype=GRB.CONTINUOUS,name="Y")
    # Binary variable indicating whether there has been a price change from time t to time t+1
    P = M1.addVars(times,vtype=GRB.BINARY,name="P")
    # Lower price at any given hour
    Pl = M1.addVars(times,lb=0.0,ub=GRB.INFINITY,vtype=GRB.CONTINUOUS,name="Pl")
    # Higher price at any given hour for consumption above threshold
    Ph = M1.addVars(times,lb=0.0,ub=GRB.INFINITY,vtype=GRB.CONTINUOUS,name="Ph")

    #defining followers variable
    # Electricity consumed at the lower price for a given consumer at a given time
    Cl = M1.addVars(customers,times,lb=0.0,ub=GRB.INFINITY,vtype=GRB.CONTINUOUS,name="Cl")
    # Electricity consumed at higher price for given consumer at a given time
    Ch = M1.addVars(customers,times,lb=0.0,ub=GRB.INFINITY,vtype=GRB.CONTINUOUS,name="Ch")
    # Demand shifted upward over existing demand by customer at given time
    Vplus = M1.addVars(customers,times,lb=0.0,ub=GRB.INFINITY,vtype=GRB.CONTINUOUS,name="Vplus")
    # Demand shifted downward over existing demand by customer at given time
    Vminus = M1.addVars(customers,times,lb=0.0,ub=GRB.INFINITY,vtype=GRB.CONTINUOUS,name="Vminus")
    # Electricity bought from rival/competitor at flat rate by consumer at given hour
    U = M1.addVars(customers,times,lb=0.0,ub=GRB.INFINITY,vtype=GRB.CONTINUOUS,name="U")

    #Defining KKT condition variables
    MUa = M1.addVars(customers,times,lb=-GRB.INFINITY,ub=GRB.INFINITY,vtype=GRB.CONTINUOUS,name="MUa")
    MUb = M1.addVars(customers,lb=-GRB.INFINITY,ub=GRB.INFINITY,vtype=GRB.CONTINUOUS,name="MUb")
    MUc = M1.addVars(customers,times,lb=0.0,ub=GRB.INFINITY,vtype=GRB.CONTINUOUS,name="MUc")
    MUd = M1.addVars(customers,times,lb=0.0,ub=GRB.INFINITY,vtype=GRB.CONTINUOUS,name="MUd")

    # Binary Variables to be used in complementatry slackness condition constraints
    roa = M1.addVars(customers,times,vtype=GRB.BINARY,name="roa")
    rob = M1.addVars(customers,times,vtype=GRB.BINARY,name="rob")
    roc = M1.addVars(customers,times,vtype=GRB.BINARY,name="roc")
    rod = M1.addVars(customers,times,vtype=GRB.BINARY,name="rod")
    roe = M1.addVars(customers,times,vtype=GRB.BINARY,name="roe")
    rof = M1.addVars(customers,times,vtype=GRB.BINARY,name="rof")
    rog = M1.addVars(customers,times,vtype=GRB.BINARY,name="rog")

    M1.update()
    #Adding Constraints:

    #leader's
    # Electricity produced at any hour is equal to the sum of electricity consumed at lower and higher prices for all the customers
    M1.addConstrs(
        (Cl.sum('*',t) + Ch.sum('*',t) - X[t] == 0 for t in range(n_t)), name="9a")
    
    # Units of electricity consumed of different fuel sources muliplied by alpha rate will give Electricity produced at that hour
    M1.addConstrs(
        (quicksum(alp_rate[s]*Y[s,t] for s in range(n_fuel)) - X[t] == 0 for t in range(n_t)), name="9b")
    
    # Difference in lower prices between any two time intervals can't be higher than a limit
    M1.addConstrs(
        (Pl[t] - Pl[t+1] <= BIGM_price*P[t] for t in range(n_t-1)), name="9d")
    M1.addConstrs(
        (Pl[t] - Pl[t+1] >= -BIGM_price*P[t] for t in range(n_t-1)), name="9d1")
    
    # Difference in higher prices between any two time intervals can't be higher than a limit
    M1.addConstrs(
        (Ph[t] - Ph[t+1] <= BIGM_price*P[t] for t in range(n_t-1)), name="9e")
    M1.addConstrs(
        (Ph[t] - Ph[t+1] >= -BIGM_price*P[t] for t in range(n_t-1)), name="9e1")
    
    # Constraint to limit total no of price changes
    M1.addConstr(
        (quicksum(P[t] for t in range(n_t)) <= caligraphy_n), name="9f")
    
    # Price at cosumption below threshold lower than price at higher consumption
    M1.addConstrs(
        (Pl[t] <= Ph[t] for t in range(n_t)), name="9g")

    #Followers
    
    # Customer demand at any hour is energy consumed at lower price plus energy consumed at higher price plus energy bought from rival and the demand shift undertaken by customer 
    M1.addConstrs(
        (Cl[k,t]+Ch[k,t]+Vminus[k,t]+U[k,t]-Vplus[k,t] == demand_profile.iloc[k,t] for k in range(n_cust) for t in range(n_t)), name="10a")
    
    # Customer demand for whole day is summation of energy cosumed at lower price plus energy at higher price plus energy from rival for all the hours of the day 
    M1.addConstrs(
        (quicksum(Cl[k,t] + Ch[k,t] + U[k,t] for t in range(n_t)) == demand[k] for k in range(n_cust)), name="10b")
    
    # Energy consumed at lower rate is lower than the threshold set by the retailer
    M1.addConstrs(
        (Cl[k,t] <= threshold for k in range(n_cust) for t in range(n_t)), name="10c")
    
    # Demand shift at any hour is lower than the demand shift capacity of the consumer for the given hour
    M1.addConstrs(
        (Vplus[k,t] <= omega_rate.iloc[k,t] for k in range(n_cust) for t in range(n_t)), name="10d")
    
    M1.update()

    #KKT Conditions
    
    #Stationary Constraints
    
    M1.addConstrs(
        (MUa[k,t] <= 0 for k in range(n_cust) for t in range(n_t)), name="11d")

    M1.addConstrs(
        (MUa[k,t] + MUb[k] <= cost_gamma[t] for k in range(n_cust) for t in range(n_t)), name="11e")
    M1.addConstrs(
        (MUa[k,t] + MUb[k] - MUc[k,t] <= Pl[t] for k in range(n_cust) for t in range(n_t)), name="11a")

    M1.addConstrs(
        (MUa[k,t] + MUb[k] <= Ph[t] for k in range(n_cust) for t in range(n_t)), name="11b")

    M1.addConstrs(
        (-MUa[k,t] - MUd[k,t] <= cost_xi.iloc[k,t] for k in range(n_cust) for t in range(n_t)), name="11c")

    
    #Complementary Slackness Condition Constraints
    
    M1.addConstrs(
        (cost_xi.iloc[k,t] + MUa[k,t] + MUd[k,t]  <= BIGM_kkt1*(1-roe[k,t]) for k in range(n_cust) for t in range(n_t)), name="11k")
    M1.addConstrs(
        (Vplus[k,t] <= omega_rate.iloc[k,t]*roe[k,t] for k in range(n_cust) for t in range(n_t)), name="11k1")
    
    M1.addConstrs(
        (0 - MUa[k,t] <= BIGM_kkt1*(1-rof[k,t]) for k in range(n_cust) for t in range(n_t)), name="11l")
    M1.addConstrs(
        (Vminus[k,t] <= demand_profile.iloc[k,t]*rof[k,t] for k in range(n_cust) for t in range(n_t)), name="11l1")
    
    
    M1.addConstrs(
        (threshold - Cl[k,t] <= threshold*(1-roa[k,t]) for k in range(n_cust) for t in range(n_t)), name="11g")
    M1.addConstrs(
        (MUc[k,t] <= BIGM_kkt1*roa[k,t] for k in range(n_cust) for t in range(n_t)), name="11g1")

    M1.addConstrs(
        (omega_rate.iloc[k,t]-Vplus[k,t] <= omega_rate.iloc[k,t]*(1-rob[k,t]) for k in range(n_cust) for t in range(n_t)), name="11h")
    M1.addConstrs(
        (MUd[k,t] <= BIGM_kkt1*rob[k,t] for k in range(n_cust) for t in range(n_t)), name="11h1")
    
    M1.addConstrs(
        (Pl[t] - MUa[k,t] - MUb[k] + MUc[k,t] <= BIGM_kkt2*(1-roc[k,t]) for k in range(n_cust) for t in range(n_t)), name="11i")
    M1.addConstrs(
        (Cl[k,t] <= threshold*roc[k,t] for k in range(n_cust) for t in range(n_t)), name="11i1")
    
    M1.addConstrs(
        (Ph[t] - MUa[k,t] - MUb[k]<= BIGM_kkt2*(1-rod[k,t]) for k in range(n_cust) for t in range(n_t)), name="11j")
    M1.addConstrs(
        (Ch[k,t] <= (demand_profile.iloc[k,t]+omega_rate.iloc[k,t]-threshold)*rod[k,t] for k in range(n_cust) for t in range(n_t)), name="11j1")
    
    
    M1.addConstrs(
        (cost_gamma[t] - MUa[k,t] - MUb[k] <= BIGM_kkt1*(1-rog[k,t]) for k in range(n_cust) for t in range(n_t)), name="11m")
    M1.addConstrs(
        (U[k,t] <= (demand_profile.iloc[k,t]+omega_rate.iloc[k,t])*rog[k,t] for k in range(n_cust) for t in range(n_t)), name="11m1")

    #setting up the objective

    M1.setObjective(
        quicksum(demand_profile.iloc[k,t]*MUa[k,t] - threshold*MUc[k,t] - omega_rate.iloc[k,t]*MUd[k,t] - cost_gamma[t]*U[k,t]
                     -cost_xi.iloc[k,t]*Vplus[k,t] for k in range(n_cust) for t in range(n_t)) +
        quicksum(demand[k]*MUb[k] for k in range(n_cust)) -
        quicksum(cost_fuel[s]*Y[s,t] for s in range(n_fuel) for t in range(n_t))-
        quicksum(emission_penalty*beta_rate[s]*Y[s,t] for s in range(n_fuel) for t in range(n_t)), GRB.MAXIMIZE)

    M1.update()
    
    M1.reset(0)
    print('--------------------------\nStarted Solving MCE Model')
    M1.optimize()
    
    M1_obj = M1.objVal
    print('Optimality Gap: {:.2f}'.format(M1.MIPGap))
    
    #--------Capturing Details of Leaders----------------
    print('\n\tGetting Results\n--------------------------')
    
    #First create a list of all variables which are required to be output
    pl = [Pl[t] for t in range(n_t)];
    ph = [Ph[t] for t in range(n_t)];
    x = [X[t] for t in range(n_t)];
    f1 = [Y[0,t] for t in range(n_t)]; #Coal usage
    f2 = [Y[1,t] for t in range(n_t)]; #Gas usage
    f3 = [Y[2,t] for t in range(n_t)]; #Oil usage
    c_l = [];
    c_h = [];
    u = []
    vplus = []
    vminus = []
    for t in range(n_t):
        c_l.append(sum(M1.getAttr(GRB.Attr.X, [Cl[k,t] for k in range(n_cust)])))
        c_h.append(sum(M1.getAttr(GRB.Attr.X, [Ch[k,t] for k in range(n_cust)])))
        u.append(sum(M1.getAttr(GRB.Attr.X, [U[k,t] for k in range(n_cust)])))
        vplus.append(sum(M1.getAttr(GRB.Attr.X, [Vplus[k,t] for k in range(n_cust)])))
        vminus.append(sum(M1.getAttr(GRB.Attr.X, [Vminus[k,t] for k in range(n_cust)])))
    
    df_leader = pd.DataFrame(index=range(n_t));
    df_leader.index.name = 'Day-hours'
    
    df_leader['Low_Price'] = M1.getAttr(GRB.Attr.X, pl);
    df_leader['High_Price'] = M1.getAttr(GRB.Attr.X, ph);
    df_leader['Elec_Generated'] = M1.getAttr(GRB.Attr.X, x);
    df_leader['Low_Price Demand'] = c_l
    df_leader['High_Price_Demand'] = c_h
    df_leader['Upward_Demand_Shifted'] = vplus
    df_leader['Downward_Demand_Shifted'] = vminus
    df_leader['Comp_Demand'] = u
    df_leader['Coal_Used'] = M1.getAttr(GRB.Attr.X, f1);
    df_leader['Coal_Emissions (lbs)'] = df_leader['Coal_Used']*data.iloc[0,4];
    df_leader['Gas_Used'] = M1.getAttr(GRB.Attr.X, f2);
    df_leader['Gas_Emissions (lbs)'] = df_leader['Gas_Used']*data.iloc[1,4];
    df_leader['Oil_Used'] = M1.getAttr(GRB.Attr.X, f3);
    df_leader['Oil_Emissions (lbs)'] = df_leader['Oil_Used']*data.iloc[2,4];
    
    #--------Capturing Details of Followers----------------
    cust = ['Cust'+str(j) for j in range(n_cust)]
    df_ch = pd.DataFrame(index=range(n_t));
    df_ch.index.name = 'Day-hours'
    
    df_cl = pd.DataFrame(index=range(n_t));
    df_cl.index.name = 'Day-hours'
    
    df_vplus = pd.DataFrame(index=range(n_t));
    df_vplus.index.name = 'Day-hours'
    
    df_vminus = pd.DataFrame(index=range(n_t));
    df_vminus.index.name = 'Day-hours'
    
    df_ukt = pd.DataFrame(index=range(n_t));
    df_ukt.index.name = 'Day-hours'
    
    for k in range(nr_cust):
        df_ch[cust[k]] = M1.getAttr(GRB.Attr.X, [Ch[k,t] for t in range(n_t)])
        df_cl[cust[k]] = M1.getAttr(GRB.Attr.X, [Cl[k,t] for t in range(n_t)])
        df_vplus[cust[k]] = M1.getAttr(GRB.Attr.X, [Vplus[k,t] for t in range(n_t)])
        df_vminus[cust[k]] = M1.getAttr(GRB.Attr.X, [Vminus[k,t] for t in range(n_t)])
        df_ukt[cust[k]] = M1.getAttr(GRB.Attr.X, [U[k,t] for t in range(n_t)])

    
    return M1_obj, df_leader.round(0), df_wh.round(0), df_wl.round(0), df_vplus.round(0), df_vminus.round(0), df_ukt.round(0)

In [ ]:
def without_emission(n_t,n_fuel,n_cust,alp_rate,fuel_supply,epsilon_rate,BIGM_price,caligraphy_n,beta_rate,
                  demand_profile,demand,threshold,omega_rate,cost_xi,cost_gamma,BIGM_kkt1,BIGM_kkt2,BIGM_kkt3,
                  cost_fuel,cost_ramp,emission_penalty,data=data_f):
    
    """Without Emission Model"""
    
    M2=Model(env=gurobi_env, name='M2')

    #Defining Leaders' variable i.e the retailer
    X = M2.addVars(times,lb=0.0,ub=GRB.INFINITY,vtype=GRB.CONTINUOUS,name="X")
    Y = M2.addVars(fuels,times,lb=0.0,ub=GRB.INFINITY,vtype=GRB.CONTINUOUS,name="Y")
    P = M2.addVars(times,vtype=GRB.BINARY,name="P")
    Pl = M2.addVars(times,lb=0.0,ub=GRB.INFINITY,vtype=GRB.CONTINUOUS,name="PIl")
    Ph = M2.addVars(times,lb=0.0,ub=GRB.INFINITY,vtype=GRB.CONTINUOUS,name="PIh")

    #defining followers variable i.e the customer
    Cl = M2.addVars(customers,times,lb=0.0,ub=GRB.INFINITY,vtype=GRB.CONTINUOUS,name="Cl")
    Ch = M2.addVars(customers,times,lb=0.0,ub=GRB.INFINITY,vtype=GRB.CONTINUOUS,name="Ch")
    Vplus = M2.addVars(customers,times,lb=0.0,ub=GRB.INFINITY,vtype=GRB.CONTINUOUS,name="Vplus")
    Vminus = M2.addVars(customers,times,lb=0.0,ub=GRB.INFINITY,vtype=GRB.CONTINUOUS,name="Vminus")
    U = M2.addVars(customers,times,lb=0.0,ub=GRB.INFINITY,vtype=GRB.CONTINUOUS,name="U")

    #Defining KKT condition variables
    MUa = M2.addVars(customers,times,lb=-GRB.INFINITY,ub=GRB.INFINITY,vtype=GRB.CONTINUOUS,name="MUa")
    MUb = M2.addVars(customers,lb=-GRB.INFINITY,ub=GRB.INFINITY,vtype=GRB.CONTINUOUS,name="MUb")
    MUc = M2.addVars(customers,times,lb=0.0,ub=GRB.INFINITY,vtype=GRB.CONTINUOUS,name="MUc")
    MUd = M2.addVars(customers,times,lb=0.0,ub=GRB.INFINITY,vtype=GRB.CONTINUOUS,name="MUd")

    #McCormic
    roa = M2.addVars(customers,times,vtype=GRB.BINARY,name="roa")
    rob = M2.addVars(customers,times,vtype=GRB.BINARY,name="rob")
    roc = M2.addVars(customers,times,vtype=GRB.BINARY,name="roc")
    rod = M2.addVars(customers,times,vtype=GRB.BINARY,name="rod")
    roe = M2.addVars(customers,times,vtype=GRB.BINARY,name="roe")
    rof = M2.addVars(customers,times,vtype=GRB.BINARY,name="rof")
    rog = M2.addVars(customers,times,vtype=GRB.BINARY,name="rog")

    M2.update()
    #Adding Constraints:

    #leader's
    M2.addConstrs(
        (Cl.sum('*',t) + Ch.sum('*',t) - X[t] == 0 for t in range(n_t)), name="9a")

    M2.addConstrs(
        (quicksum(alp_rate[s]*Y[s,t] for s in range(n_fuel)) - X[t] == 0 for t in range(n_t)), name="9b")


    M2.addConstrs(
        (Pl[t] - Pl[t+1] <= BIGM_price*P[t] for t in range(n_t-1)), name="9d")
    M2.addConstrs(
        (Pl[t] - Pl[t+1] >= -BIGM_price*P[t] for t in range(n_t-1)), name="9d1")

    M2.addConstrs(
        (Ph[t] - Ph[t+1] <= BIGM_price*P[t] for t in range(n_t-1)), name="9e")
    M2.addConstrs(
        (Ph[t] - Ph[t+1] >= -BIGM_price*P[t] for t in range(n_t-1)), name="9e1")

    M2.addConstr(
        (quicksum(P[t] for t in range(n_t)) <= caligraphy_n), name="9f")

    M2.addConstrs(
        (Pl[t] <= Ph[t] for t in range(n_t)), name="9g")

    #Followers
    M2.addConstrs(
        (Cl[k,t]+Ch[k,t]+Vminus[k,t]+U[k,t]-Vplus[k,t] == demand_profile.iloc[k,t] for k in range(n_cust) for t in range(n_t)), name="10a")

    M2.addConstrs(
        (quicksum(Cl[k,t] + Ch[k,t] + U[k,t] for t in range(n_t)) == demand[k] for k in range(n_cust)), name="10b")

    M2.addConstrs(
        (Cl[k,t] <= threshold for k in range(n_cust) for t in range(n_t)), name="10c")

    M2.addConstrs(
        (Vplus[k,t] <= omega_rate.iloc[k,t] for k in range(n_cust) for t in range(n_t)), name="10d")
    
    M2.update()

    #KKT Conditions
    
    #Stationary Constraints
    M2.addConstrs(
        (MUa[k,t] + MUb[k] - MUc[k,t] <= Pl[t] for k in range(n_cust) for t in range(n_t)), name="11a")

    M2.addConstrs(
        (MUa[k,t] + MUb[k] <= Ph[t] for k in range(n_cust) for t in range(n_t)), name="11b")

    M2.addConstrs(
        (-MUa[k,t] - MUd[k,t] <= cost_xi.iloc[k,t] for k in range(n_cust) for t in range(n_t)), name="11c")

    M2.addConstrs(
        (MUa[k,t] <= 0 for k in range(n_cust) for t in range(n_t)), name="11d")

    M2.addConstrs(
        (MUa[k,t] + MUb[k] <= cost_gamma[t] for k in range(n_cust) for t in range(n_t)), name="11e")
    
    #Complementary Slackness Condition Constraints
    M2.addConstrs(
        (threshold - Cl[k,t] <= threshold*(1-roa[k,t]) for k in range(n_cust) for t in range(n_t)), name="11g")
    M2.addConstrs(
        (MUc[k,t] <= BIGM_kkt1*roa[k,t] for k in range(n_cust) for t in range(n_t)), name="11g1")

    M2.addConstrs(
        (omega_rate.iloc[k,t]-Vplus[k,t] <= omega_rate.iloc[k,t]*(1-rob[k,t]) for k in range(n_cust) for t in range(n_t)), name="11h")
    M2.addConstrs(
        (MUd[k,t] <= BIGM_kkt1*rob[k,t] for k in range(nr_cust) for t in range(nr_t)), name="11h1")
    
    M2.addConstrs(
        (Pl[t] - MUa[k,t] - MUb[k] + MUc[k,t] <= BIGM_kkt2*(1-roc[k,t]) for k in range(n_cust) for t in range(n_t)), name="11i")
    M2.addConstrs(
        (Cl[k,t] <= threshold*roc[k,t] for k in range(n_cust) for t in range(n_t)), name="11i1")
    
    M2.addConstrs(
        (Ph[t] - MUa[k,t] - MUb[k]<= BIGM_kkt2*(1-rod[k,t]) for k in range(n_cust) for t in range(n_t)), name="11j")
    M2.addConstrs(
        (Ch[k,t] <= (demand_profile.iloc[k,t]+omega_rate.iloc[k,t]-threshold)*rod[k,t] for k in range(n_cust) for t in range(n_t)), name="11j1")
    
    M2.addConstrs(
        (cost_xi.iloc[k,t] + MUa[k,t] + MUd[k,t]  <= BIGM_kkt1*(1-roe[k,t]) for k in range(n_cust) for t in range(n_t)), name="11k")
    M2.addConstrs(
        (Vplus[k,t] <= omega_rate.iloc[k,t]*roe[k,t] for k in range(n_cust) for t in range(n_t)), name="11k1")
    
    M2.addConstrs(
        (0 - MUa[k,t] <= BIGM_kkt1*(1-rof[k,t]) for k in range(n_cust) for t in range(n_t)), name="11l")
    M2.addConstrs(
        (Vminus[k,t] <= demand_profile.iloc[k,t]*rof[k,t] for k in range(n_cust) for t in range(n_t)), name="11l1")
    
    M2.addConstrs(
        (cost_gamma[t] - MUa[k,t] - MUb[k] <= BIGM_kkt1*(1-rog[k,t]) for k in range(n_cust) for t in range(n_t)), name="11m")
    M2.addConstrs(
        (U[k,t] <= (demand_profile.iloc[k,t]+omega_rate.iloc[k,t])*rog[k,t] for k in range(n_cust) for t in range(n_t)), name="11m1")

    #setting up the objective

    M2.setObjective(
        quicksum(demand_profile.iloc[k,t]*MUa[k,t] - threshold*MUc[k,t] - omega_rate.iloc[k,t]*MUd[k,t] - cost_gamma[t]*U[k,t]
                     -cost_xi.iloc[k,t]*Vplus[k,t] for k in range(n_cust) for t in range(n_t)) +
        quicksum(demand[k]*MUb[k] for k in range(n_cust)) -
        quicksum(cost_fuel[s]*Y[s,t] for s in range(n_fuel) for t in range(n_t)), GRB.MAXIMIZE)

    M2.update()
    
    M2.reset(0)
    print(f'--------------------------\nStarted Solving MWCE Model {psi_rate}')
    M2.optimize()
    
    M2_obj = M2.objVal
    
    
    #--------Capturing Details of Leaders----------------
    
    print('\n\tGetting Results\n--------------------------')
    
    # create a list of all variables which are required to be output
    pl = [Pl[t] for t in range(n_t)];
    ph = [Ph[t] for t in range(n_t)];
    x = [X[t] for t in range(n_t)];
    f1 = [Y[0,t] for t in range(n_t)]; #Coal usage
    f2 = [Y[1,t] for t in range(n_t)]; #Gas usage
    f3 = [Y[2,t] for t in range(n_t)]; #Oil usage
    c_l = [];
    c_h = [];
    u = []
    vplus = []
    vminus = []
    for t in range(nr_t):
        c_l.append(sum(M2.getAttr(GRB.Attr.X, [Cl[k,t] for k in range(nr_cust)])))
        c_h.append(sum(M2.getAttr(GRB.Attr.X, [Ch[k,t] for k in range(nr_cust)])))
        u.append(sum(M2.getAttr(GRB.Attr.X, [U[k,t] for k in range(nr_cust)])))
        vplus.append(sum(M2.getAttr(GRB.Attr.X, [Vplus[k,t] for k in range(nr_cust)])))
        vminus.append(sum(M2.getAttr(GRB.Attr.X, [Vminus[k,t] for k in range(nr_cust)])))
    
    df_leader = pd.DataFrame(index=range(nr_t));
    df_leader.index.name = 'Day-hours'
    
    df_leader['Low_Price'] = M2.getAttr(GRB.Attr.X, pl);
    df_leader['High_Price'] = M2.getAttr(GRB.Attr.X, ph);
    df_leader['Elec_Generated'] = M2.getAttr(GRB.Attr.X, x);
    df_leader['Low_Price Demand'] = c_l
    df_leader['High_Price_Demand'] = c_h
    df_leader['Upward_Demand_Shifted'] = vplus
    df_leader['Downward_Demand_Shifted'] = vminus
    df_leader['Comp_Demand'] = u
    df_leader['Coal_Used'] = M2.getAttr(GRB.Attr.X, f1);
    df_leader['Coal_Emissions (lbs)'] = df_leader['Coal_Used']*data.iloc[0,4];
    df_leader['Gas_Used'] = M2.getAttr(GRB.Attr.X, f2);
    df_leader['Gas_Emissions (lbs)'] = df_leader['Gas_Used']*data.iloc[1,4];
    df_leader['Oil_Used'] = M2.getAttr(GRB.Attr.X, f3);
    df_leader['Oil_Emissions (lbs)'] = df_leader['Oil_Used']*data.iloc[2,4];
    
    #--------Capturing Details of Followers----------------
    cust = ['Cust'+str(j) for j in range(n_cust)]
    df_ch = pd.DataFrame(index=range(n_t));
    df_ch.index.name = 'Day-hours'
    
    df_cl = pd.DataFrame(index=range(n_t));
    df_cl.index.name = 'Day-hours'
    
    df_vplus = pd.DataFrame(index=range(n_t));
    df_vplus.index.name = 'Day-hours'
    
    df_vminus = pd.DataFrame(index=range(n_t));
    df_vminus.index.name = 'Day-hours'
    
    df_ukt = pd.DataFrame(index=range(n_t));
    df_ukt.index.name = 'Day-hours'
    
    for k in range(nr_cust):
        df_ch[cust[k]] = M2.getAttr(GRB.Attr.X, [Ch[k,t] for t in range(n_t)])
        df_cl[cust[k]] = M2.getAttr(GRB.Attr.X, [Cl[k,t] for t in range(n_t)])
        df_vplus[cust[k]] = M2.getAttr(GRB.Attr.X, [Vplus[k,t] for t in range(n_t)])
        df_vminus[cust[k]] = M2.getAttr(GRB.Attr.X, [Vminus[k,t] for t in range(n_t)])
        df_ukt[cust[k]] = M2.getAttr(GRB.Attr.X, [U[k,t] for t in range(n_t)])
    
    return M2_obj, df_leader.round(0), df_wh.round(0), df_wl.round(0), df_vplus.round(0), df_vminus.round(0), df_ukt.round(0)